# Data Preparation
This part consist of data reading, analyzing, cleaning and saving for further work

In [1]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, TimestampType

In [2]:
# creating the schema for dataset
schema = StructType([ \
    StructField("event_time",TimestampType(),True),\
    StructField("event_type",StringType(),False),\
    StructField("product_id",IntegerType(),True),\
    StructField("category_id", LongType(), True),\
    StructField("category_code", StringType(), True),\
    StructField("brand", StringType(), True),\
    StructField("price", DoubleType(), True),\
    StructField("user_id", IntegerType(), True),\
    StructField("user_session", StringType(), True),\
  ])

In [3]:
# Read the dataset
df=spark.read.csv('local_part.csv', schema=schema, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [5]:
df.show()

+-------------------+----------+----------+-------------------+--------------------+---------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|    brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+---------+------+---------+--------------------+
|2019-10-01 01:00:01|      view|   1307067|2053013558920217191|  computers.notebook|   lenovo|251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 01:00:20|      view|   1003306|2053013555631882655|electronics.smart...|    apple|588.77|555446831|6ec635da-ea15-4a5...|
|2019-10-01 01:00:25|      view|  27500014|2053013554692358509|                null|  redmond| 37.98|555217733|74d40a28-41f9-432...|
|2019-10-01 01:00:31|      view|  28718079|2053013565362668491|  apparel.shoes.keds|  respect| 66.67|545323115|75fb5d0c-e907-429...|
|2019-10-01 01:00:31|      view|   3900746|2053013552326770905|applia

In [6]:
# number of unic users
df.groupBy('user_id').count().distinct().count()

1390837

In [7]:
# Users events count
df.groupBy('user_id').count().orderBy('count',ascending=False).show(10,False)

+---------+-----+
|user_id  |count|
+---------+-----+
|512475445|806  |
|512365995|421  |
|526731152|306  |
|513021392|293  |
|512505687|283  |
|546270188|258  |
|546159478|257  |
|516308435|253  |
|514649263|240  |
|551211823|238  |
+---------+-----+
only showing top 10 rows



In [8]:
# brands events count
df.groupBy('brand').count().orderBy('count',ascending=False).show(10,False)

+-------+------+
|brand  |count |
+-------+------+
|null   |610994|
|samsung|527583|
|apple  |411810|
|xiaomi |308436|
|huawei |110888|
|lucente|65673 |
|lg     |56602 |
|bosch  |56118 |
|oppo   |48640 |
|sony   |45535 |
+-------+------+
only showing top 10 rows



In [9]:
# events count
df.groupBy('event_type').count().orderBy('count',ascending=False).show(10,False)

+----------+-------+
|event_type|count  |
+----------+-------+
|view      |4078750|
|cart      |92702  |
|purchase  |74196  |
+----------+-------+



In [10]:
# for using Collaborative Filtering for Implicit Feedback Datasets,
# convert event_type to raiting such us purchase = 1, view and chart = 0
# so if user bought item, he "rated" it
dictionary = {"purchase": "1", "view": "0", "cart": "0"}
df2 = df.na.replace(dictionary,"event_type")

# cast to integer
df_data = df2.select(df2["event_type"].cast(IntegerType()), df2["user_id"], df2["product_id"]) 
df_data.show()

/opt/spark/python/pyspark/sql/dataframe.py:1894: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


+----------+---------+----------+
|event_type|  user_id|product_id|
+----------+---------+----------+
|         0|550050854|   1307067|
|         0|555446831|   1003306|
|         0|555217733|  27500014|
|         0|545323115|  28718079|
|         0|555444559|   3900746|
|         0|515454339|  12712064|
|         0|551377651|   1003141|
|         0|519885473|   4100126|
|         0|555447577|  28717211|
|         0|512558158|   1004659|
|         0|525856698|  26500144|
|         0|544648245|   4300070|
|         0|513457407|   1004792|
|         0|550050854|   1306631|
|         0|514336739|   1004321|
|         0|537918940|   1004545|
|         0|555447570|  28715758|
|         0|519885473|   4100274|
|         0|516896785|   1004792|
|         0|555447748|  26201000|
+----------+---------+----------+
only showing top 20 rows



In [14]:
# drop rows with nulls
df_data = df_data.na.drop()

In [15]:
# save to parquet file for further work
df_data.write.mode("overwrite").parquet("preparedDataset.parquet")

In [16]:
! ls -la

total 1115156
drwxrwxr-x  6 big big      4096 May 21 15:18 .
drwxr-xr-x 33 big big      4096 May 12 14:59 ..
-rw-rw-r--  1 big big     13414 May 21 15:16 Data_Preparation.ipynb
drwxrwxr-x  8 big big      4096 May 21 14:34 .git
-rw-rw-r--  1 big big        22 May 21 14:33 .gitignore
drwxrwxr-x  2 big big      4096 May 21 14:41 .ipynb_checkpoints
-rw-rw-r--  1 big big 570895494 May  8 17:13 local_part.csv
-rw-r--r--  1 big big 171002105 May 11 21:13 local_test.csv
-rw-r--r--  1 big big 399893389 May 11 21:13 local_train.csv
-rw-rw-r--  1 big big      3716 May 12 22:05 partial_data.ipynb
drwxr-xr-x  2 big big      4096 May 21 15:15 preparedDataset.parquet
-rw-rw-r--  1 big big      3566 May 21 15:18 Recommendation_model.ipynb
-rw-rw-r--  1 big big     53988 May 21 13:06 Recommender.ipynb
drwxr-xr-x  2 big big      4096 May 21 10:21 userSubsetRecs.parquet
